In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
?Ridge

In [ ]:
df = pd.read_csv('/content/NPPE1_ModelBuilding3.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.087472,0.002829,1.768235,0.188396,0.117071,0.148148,0.478165,0.720443,0.967195,0.281563,0.428571,0.959596,1.0,0.0,23.358097
1,0.378379,0.022079,1.115629,0.091974,0.066089,0.851852,0.911759,0.785321,0.885001,0.424648,0.285714,0.868687,0.0,1.0,17.268768
2,0.066901,0.003828,-0.536262,0.221188,0.255671,0.296296,0.228024,0.406472,0.980184,0.274376,0.428571,0.767677,0.0,1.0,27.776974
3,0.140645,0.011132,1.323366,0.422514,0.153103,0.148148,0.410679,0.200319,0.861371,0.305006,0.142857,0.848485,0.0,1.0,16.121960
4,0.144225,0.204918,-0.930790,0.148694,0.177490,0.259259,0.146832,0.111429,0.983448,0.286322,0.285714,0.616162,1.0,0.0,23.129426


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('14', axis = 1), df['14'], test_size=0.3, random_state=42)

In [ ]:
X_train.shape

(2800, 14)

In [ ]:
model = Ridge(alpha=10, solver='saga', tol=1e-4, random_state=42)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
model.fit(X_train, y_train)

Ridge(alpha=10, random_state=42, solver='saga')

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
?Ridge

In [ ]:
r2_score(y_test, y_pred)

0.6613547575262211

In [ ]:
coefs = np.abs(model.coef_)
print(np.argmax(coefs))
print(np.argmin(coefs))

9
0


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor

In [ ]:
param_grid = {'penalty':['l1', 'l2'],
              'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
              'tol': [1e-4, 1e-3, 1e-2, 1e-1]}

clf = GridSearchCV(estimator=SGDRegressor(random_state=42), param_grid=param_grid, cv=5, scoring='neg_mean_absolute_error')


In [ ]:
clf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SGDRegressor(random_state=42),
             param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.01, 0.1],
                         'penalty': ['l1', 'l2'],
                         'tol': [0.0001, 0.001, 0.01, 0.1]},
             scoring='neg_mean_absolute_error')

In [ ]:
clf.best_params_

{'alpha': 0.001, 'penalty': 'l2', 'tol': 0.0001}

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
?mean_absolute_error

In [ ]:
mean_absolute_error(y_test, y_pred)

3.8131121797994014

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso
?PCA

In [ ]:
pipe = Pipeline([("pca", PCA()),
                 ('lasso', Lasso())])

param_grid = {
    'pca__n_components': [0.9, 0.95],
    'lasso__alpha': [10, 1, 0.01, 0.001]
}

gs = GridSearchCV(pipe, param_grid, scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)

In [ ]:
gs.fit(X_train, y_train)
y_pred = gs.predict(X_test)
r2_score(y_test, y_pred)

0.6288625430197575

In [ ]:
best_pca = gs.best_estimator_.named_steps['pca']
eig_pc1 = best_pca.explained_variance_
eig_pc1

array([1.16350757, 0.15109745, 0.11321585, 0.07019734, 0.03866388,
       0.02770669, 0.02515772])

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
pipe = Pipeline([('pf', PolynomialFeatures(interaction_only=False, degree=2)),
                 ('lasso', Lasso(alpha=1, warm_start=True, random_state=0))])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
r2_score(y_test, y_pred)

0.157678032410551

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [ ]:
selector = RFE(LinearRegression(), n_features_to_select=X_train.shape[1] - 1)
selector.fit(X_train, y_train)
selector.support_

array([ True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])